In [0]:
import pandas as pd
import numpy as np
df_tr = pd.read_csv('selected_48_tr.csv')
df_te = pd.read_csv('selected_48_te.csv')

In [0]:
df_train = df_tr
df_test = df_te

In [0]:
print(df_train.shape)
print(df_test.shape)

(68381, 50)
(29307, 50)


In [0]:
## calculate an_loss in both dataset
df_train['an_loss'] = df_train['ncat']/df_train['ee']
df_test['an_loss'] = df_test['ncat']/df_test['ee']
print(df_train.head())
print(df_test.head())

   oldmodel  cr_robb_s_1y  cr_agg_r_1y  ...  ncat      ee  an_loss
0    339.94      1.000000     0.794118  ...     0  1.0000      0.0
1    248.77      1.000000     0.923077  ...     0  0.0438      0.0
2    523.65     11.006402     0.717986  ...     0  0.7973      0.0
3    213.16    290.000000     0.730134  ...     0  1.0000      0.0
4    214.90      1.000000     0.642857  ...     0  1.0000      0.0

[5 rows x 51 columns]
   oldmodel  cr_robb_s_1y  cr_agg_r_1y  ...  ncat      ee  an_loss
0    252.23     22.000000     0.674699  ...     0  0.3507      0.0
1    318.18    102.000000     0.619469  ...     0  0.0027      0.0
2    414.08      0.000000     0.769231  ...     0  0.1776      0.0
3    590.09     11.500701     0.717986  ...     0  1.0000      0.0
4    214.92     11.000000     0.823899  ...     0  1.0000      0.0

[5 rows x 51 columns]


In [0]:
df_train = df_train.loc[:,df_train.columns != 'ee']
#df_train = df_train.loc[:,df_train.columns != 'oldmodel']
df_train = df_train.loc[:,df_train.columns != 'ncat']
df_test = df_test.loc[:,df_test.columns != 'ee']
#df_test = df_test.loc[:,df_test.columns != 'oldmodel']
df_test = df_test.loc[:,df_test.columns != 'ncat']

In [0]:
#Combined both Train and Test Data set to do preprocessing together # and set flag for both as well
df_train['Type'] = 1 
df_test['Type'] = 0
fullData = pd.concat([df_train,df_test],axis=0)

In [0]:
fullData.head()
fullData.shape
fullData.columns

Index(['oldmodel', 'cr_robb_s_1y', 'cr_agg_r_1y', 'cr_vio_s_3y', 'cr_bur_s_3y',
       'cr_vio_r_st_3y', 'cr_bur_r_st_3y', 'cr_mtv_r_st_3y', 'cr_bur_r_3y',
       'cr_lacy_r_3y', 'cr_mtv_r_3y', 'cr_robb_s_5y', 'cr_agg_s_5y',
       'cr_ppty_s_5y', 'cr_vio_r_st_5y', 'cr_lacy_r_st_5y', 'cr_rape_s_10y',
       'cr_lacy_s_10y', 'cr_rape_r_st_10y', 'cr_bur_r_st_10y',
       'cr_mtv_r_st_10y', 'cr_vio_r_10y', 'cr_ppty_r_10y', 'cr_murd_r_10y',
       'cr_agg_r_10y', 'cr_lacy_r_10y', 'cr_rape_s_20y', 'cr_bur_s_20y',
       'cr_rape_r_st_20y', 'cr_bur_r_st_20y', 'cr_lacy_r_st_20y',
       'cr_vio_r_20y', 'cr_ppty_r_20y', 'cr_lacy_r_20y', 'cr_mtv_r_20y',
       'ltg_sp_strikes_3y', 'ltg_fa_events_3y', 'ltg_strikes_5y',
       'ltg_sm_strikes_5y', 'ltg_wt_strikes_5y', 'ltg_wt_events_5y',
       'ltg_strikes_10y', 'ltg_events_10y', 'ltg_sm_events_10y',
       'ltg_wt_strikes_10y', 'ltg_st_strike_event_10y', 'ltg_nt_events_10y',
       'ltg_sp_strike_event_20y', 'an_loss', 'Type'],
      dtype='obj

In [0]:
## Identify features and target
#features_col = list(df_train0.columns)
flag_col= ['Type']
target_col = ["an_loss"]

In [0]:
#Creata a validation set
from sklearn.model_selection import train_test_split
train=fullData.loc[fullData.Type == 1]
test = fullData.loc[fullData.Type == 0]
features=list(set(list(fullData.columns))-set(target_col)-set(flag_col))
## Normalized features data
# fullData[features] = fullData[features]/fullData[features].max()

X = train[features].values
y = train[target_col].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, random_state=42)

## Model Building using Deep Learning Model (Keras)

In [0]:
# Define model
import random
random.seed(42)
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
model = Sequential()
model.add(Dense(100, input_dim=48, activation= "relu"))
model.add(Dense(50, activation= 'relu'))
model.add(Dense(1))
model.summary() #Print model Summary

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               4900      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 51        
Total params: 10,001
Trainable params: 10,001
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compile model
#model.compile(loss= "mean_squared_error" , optimizer="Adamax", metrics=["mean_squared_error"])
model.compile(loss= "mse" , optimizer="adam", metrics=["mse"])

In [0]:
# Fit Model
model.fit(X_train, y_train, epochs=3, batch_size=33)

Epoch 1/3
1451/1451 [==============================] - 5s 3ms/step - loss: 3429727744.0000 - mse: 3430939648.0000
Epoch 2/3
1451/1451 [==============================] - 5s 3ms/step - loss: 3432653824.0000 - mse: 3433865984.0000
Epoch 3/3
1451/1451 [==============================] - 5s 3ms/step - loss: 3431179264.0000 - mse: 3432398080.0000


## Fit nn in-sample validation 

In [0]:
from sklearn.metrics import mean_squared_error

model.fit(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))

Epoch 1/3
1496/1496 [==============================] - 6s 4ms/step - loss: 3431397376.0000 - mse: 3431825920.0000 - val_loss: 297629152.0000 - val_mse: 298047328.0000
Epoch 2/3
1496/1496 [==============================] - 6s 4ms/step - loss: 3432369920.0000 - mse: 3432799744.0000 - val_loss: 298028032.0000 - val_mse: 298443808.0000
Epoch 3/3
1496/1496 [==============================] - 6s 4ms/step - loss: 3430427648.0000 - mse: 3430852608.0000 - val_loss: 315636768.0000 - val_mse: 315985376.0000


## Test NN in test

### put the result into one dataframe

In [0]:
#Select the independent variables for test dataset
X_test = test[features].values

#Prediction using Neural Network
y_test_nn = model.predict(X_test)

## need to change the raw df_test dataset
ee_col_te = df_te['ee']
ncat_te = df_te['ncat']

ee_te = pd.DataFrame(ee_col_te)
ncat_te = pd.DataFrame(ncat_te)
pred_test_df = pd.DataFrame(y_test_nn)

nn_test_df = pd.concat([ncat_te, ee_te, pred_test_df], axis=1)

nn_test_df.to_csv('test31_df.csv',index = False)


## Save train pred to csv

In [0]:
## predict in training set
tr = train[features].values
pred_train = model.predict(tr)

## put ee and ncat and pred to csv
ee_col_tr = df_tr['ee']
ncat_tr = df_tr['ncat']
ee_tr = pd.DataFrame(ee_col_tr)
ncat_tr = pd.DataFrame(ncat_tr)
pred_train_df = pd.DataFrame(pred_train)
nn_train_df = pd.concat([ncat_tr, ee_tr, pred_train_df], axis=1)
nn_train_df.to_csv('train31_df.csv',index = False)

## evaluate the result in validation dataset(Final_valid)

### Load the validation daset daset

In [0]:
df_valid = pd.read_csv('Validation_34.csv')
## cap in 300000
df_valid['ncat'] = np.where(df_valid['ncat'] > 300000, 300000, df_valid['ncat'])

## predit in the validation set

In [0]:
## predict in validation set
val = df_valid[features].values
pred_val = model.predict(val)

## put ee and ncat and pred to csv
ee_col_val = df_valid['ee']
ncat_val = df_valid['ncat']
ee_val = pd.DataFrame(ee_col_val)
ncat_val = pd.DataFrame(ncat_val)
pred_val_df = pd.DataFrame(pred_val)
nn_valid_df = pd.concat([ncat_val, ee_val, pred_val_df], axis=1)
nn_valid_df.to_csv('valid7_df.csv',index = False)

### test in full_train and full_test

In [0]:
## load full train data
df_full_tr = pd.read_csv('Train_full_34.csv')

In [0]:
## predict in full_train set
full_tr = df_full_tr[features].values
pred_full_tr = model.predict(full_tr)

## put ee and ncat and pred to csv
ee_col_full_tr = df_full_tr['ee']
ncat_full_tr = df_full_tr['ncat']
ee_full_tr = pd.DataFrame(ee_col_full_tr)
ncat_full_tr = pd.DataFrame(ncat_full_tr)
pred_full_tr_df = pd.DataFrame(pred_full_tr)
nn_full_tr_df = pd.concat([ncat_full_tr, ee_full_tr, pred_full_tr_df], axis=1)
nn_full_tr_df.to_csv('full_tr7_df.csv',index = False)

## Test in full_train dataset

In [0]:
## load full test dataset 
df_full_te = pd.read_csv('Test_full_34.csv')

In [0]:
## predict in full_test set
full_te = df_full_te[features].values
pred_full_te = model.predict(full_te)

## put ee and ncat and pred to csv
ee_col_full_te = df_full_te['ee']
ncat_full_te = df_full_te['ncat']
ee_full_te = pd.DataFrame(ee_col_full_te)
ncat_full_te = pd.DataFrame(ncat_full_te)
pred_full_te_df = pd.DataFrame(pred_full_te)
nn_full_te_df = pd.concat([ncat_full_te, ee_full_te, pred_full_te_df], axis=1)
nn_full_te_df.to_csv('full_te7_df.csv',index = False)